In [4]:
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Đường dẫn đến tập dữ liệu
data_train = pd.read_csv('EPL23-24.csv')
data_train.fillna(0)

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA
0,E0,11/08/2023,20:00,Burnley,Man City,0,3,A,0,2,...,2.28,1.50,1.95,1.98,1.95,1.97,0.00,0.00,1.92,1.95
1,E0,12/08/2023,12:30,Arsenal,Nott'm Forest,2,1,H,2,0,...,2.63,-2.00,1.95,1.98,1.93,1.97,2.01,2.09,1.95,1.92
2,E0,12/08/2023,15:00,Bournemouth,West Ham,1,1,D,0,0,...,2.12,0.00,2.02,1.91,2.01,1.92,2.06,1.96,1.96,1.91
3,E0,12/08/2023,15:00,Brighton,Luton,4,1,H,1,0,...,2.48,-1.75,2.01,1.92,2.00,1.91,2.14,1.93,2.00,1.86
4,E0,12/08/2023,15:00,Everton,Fulham,0,1,A,0,0,...,1.71,-0.25,2.06,1.87,2.04,1.88,2.08,1.99,1.98,1.88
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,E0,19/05/2024,16:00,Crystal Palace,Aston Villa,5,0,H,2,0,...,2.78,-0.75,1.73,2.08,1.78,2.16,1.93,2.21,1.81,2.05
376,E0,19/05/2024,16:00,Liverpool,Wolves,2,0,H,2,0,...,5.17,-2.75,2.07,1.86,2.04,1.85,2.10,1.89,2.04,1.82
377,E0,19/05/2024,16:00,Luton,Fulham,2,4,A,1,2,...,2.69,0.25,2.00,1.93,1.99,1.93,2.02,1.94,1.96,1.91
378,E0,19/05/2024,16:00,Man City,West Ham,3,1,H,2,1,...,4.82,-3.00,2.03,1.90,1.99,1.90,2.05,1.99,1.96,1.91


In [ ]:

# Chuyển đổi cột Date sang định dạng datetime
data_test_full['Date'] = pd.to_datetime(data_test_full['Date'], format='%d/%m/%Y')

# Chia dữ liệu test thành hai phần dựa trên thời gian thi đấu
data_history = data_test_full[data_test_full['Date'] < '2024-02-01']
data_test = data_test_full[data_test_full['Date'] >= '2024-02-01']

# Chọn các cột đặc trưng và cột mục tiêu
features = ['HTHG', 'HTAG', 'HS', 'AS', 'HST', 'AST', 'HC', 'AC', 'HF', 'AF', 'HY', 'AY', 'HR', 'AR']
target = 'FTR'
columns_to_halve = ['HS', 'AS', 'HST', 'AST', 'HC', 'AC', 'HF', 'AF', 'HY', 'AY', 'HR', 'AR']

# Chia nửa giá trị cho data_train
data_train[columns_to_halve] = data_train[columns_to_halve].div(2)

# Chia nửa giá trị cho data_test
data_test[columns_to_halve] = data_test[columns_to_halve].div(2)
# Mã hóa nhãn mục tiêu thành số (H->1, D->0, A->-1)
data_train[target] = data_train[target].map({'H': 1, 'D': 0, 'A': -1})
data_test[target] = data_test[target].map({'H': 1, 'D': 0, 'A': -1})
data_history[target] = data_history[target].map({'H': 1, 'D': 0, 'A': -1})

# Tính tỷ lệ thắng của 10 trận gần nhất cho mỗi đội
def calculate_win_rate(team, history, n_games=10):
    recent_matches = history[(history['HomeTeam'] == team) | (history['AwayTeam'] == team)].tail(n_games)
    wins = sum((recent_matches['HomeTeam'] == team) & (recent_matches['FTR'] == 1)) + \
           sum((recent_matches['AwayTeam'] == team) & (recent_matches['FTR'] == -1))
    return wins / n_games

# Thêm cột tỷ lệ thắng cho đội nhà và đội khách
data_train['HomeWinRate'] = data_train['HomeTeam'].apply(lambda x: calculate_win_rate(x, data_history))
data_train['AwayWinRate'] = data_train['AwayTeam'].apply(lambda x: calculate_win_rate(x, data_history))
data_test['HomeWinRate'] = data_test['HomeTeam'].apply(lambda x: calculate_win_rate(x, data_history))
data_test['AwayWinRate'] = data_test['AwayTeam'].apply(lambda x: calculate_win_rate(x, data_history))

# Thêm cột đánh dấu đội thuộc nhóm Big Six
big_six = ['Liverpool', 'Tottenham', 'Manchester City', 'Man United', 'Chelsea', 'Arsenal']
data_train['HomeBigSix'] = data_train['HomeTeam'].apply(lambda x: 1 if x in big_six else 0)
data_train['AwayBigSix'] = data_train['AwayTeam'].apply(lambda x: 1 if x in big_six else 0)
data_test['HomeBigSix'] = data_test['HomeTeam'].apply(lambda x: 1 if x in big_six else 0)
data_test['AwayBigSix'] = data_test['AwayTeam'].apply(lambda x: 1 if x in big_six else 0)

# Thêm các tính năng mới vào danh sách các cột đặc trưng
features.extend(['HomeWinRate', 'AwayWinRate', 'HomeBigSix', 'AwayBigSix'])

# Tạo dữ liệu huấn luyện và kiểm tra
X_train = data_train[features].values
y_train = data_train[target].values
X_test = data_test[features].values
y_test = data_test[target].values

# Huấn luyện mô hình SVM Classifier
svm_classifier = SVC(kernel='linear', probability=True, random_state=42)
svm_classifier.fit(X_train, y_train)

# Dự đoán trên tập dữ liệu kiểm tra
predictions = svm_classifier.predict_proba(X_test)[:, 1]

# Đánh giá mô hình
y_pred = svm_classifier.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred), '\n')

# Tạo DataFrame kết quả dự đoán
predictions_df = pd.DataFrame(columns=['HomeTeam', 'AwayTeam', 'Prediction'])
predictions_df['HomeTeam'] = data_test['HomeTeam']
predictions_df['AwayTeam'] = data_test['AwayTeam']
predictions_df['FTR'] = data_test['FTR']
predictions_df['dudoan'] = y_pred
predictions_df['Prediction'] = predictions

# In ra DataFrame
print(predictions_df)